In [1]:
import numpy as np
import re
import functools

with open("input.txt") as f:
    p = r'([a-z][a-z][a-z]) ([a-z]) ?(.*)'
    data = [re.match(p, x) for x in f.read().split('\n')]

#[print(x.groups()) for x in data]
":)"

':)'

In [2]:
#  # used to find what the commands were doing

# model_number = iter(list("000000000000000"))
# ops = {'inp': '*0+int(next(model_number)))#', 'add': '+', 
#        'mul': '*', 'div': '/', 'mod': '%', 'eql': '=='}
# w, x, y, z = 0,0,0,0
# commands = []
# lstind = -1
# for line in data:
#     command = f"{line.group(2)} = int({line.group(2)} {ops[line.group(1)]} {line.group(3)})"
#     exec(command)
#     #print(f"{command:40s} | {w}, {x}, {y}, {z}")
#     if line.group(1) == 'inp':
#         lstind += 0
#         commands.append([])
#     commands[lstind].append(line[0])
# for i in range(len(commands)):
#     for j in range(len(commands[0])):
#         try:
#             print(f"{commands[i][j]:9s}", end='|')
#         except:
#             pass
#     print()

In [3]:
def test_model(number):
    model_number = iter(list(str(number)))
    ops = {'inp': '*0+int(next(model_number)))#', 'add': '+', 
       'mul': '*', 'div': '/', 'mod': '%', 'eql': '=='}
    w, x, y, z = 0,0,0,0

    for line in data:
        command = f"{line.group(2)} = int({line.group(2)} {ops[line.group(1)]} {line.group(3)})"
        exec(command)
    return w, x, y, z

Equations to solve:

    w5  + k5  + c6  = w6
    w7  + k7  + c8  = w8
    w4  + k4  + c9  = w9
    w10 + k10 + c11 = w11
    w3  + k3  + c12 = w12
    w2  + k2  + c13 = w13
    w1  + k1  + c14 = w14

where:
- wi is the ith number of w (so w1 is most significant, w14 least)
- ki is the ith constant that gets added to y towards the end of one cycle (command 16 of 18 command cycle)
- ci is the ith constant that gets added to x towards the start of one cycle (command 6 of 18 command cycle)


In [4]:
def get_constants(data):
    '''find the constants, and the indexes that are relevant to create the above set of equations'''
    d = [x[3] for x in data]
    cycles = np.array(d).reshape(-1, 18)
    c = cycles[:,  5].astype(int)
    k = cycles[:, 15].astype(int)
    a = cycles[:,  4].astype(int)
    rhsi = np.argwhere(a==26).flatten()
    lhsi_not_sorted = np.argwhere(a != 26).flatten()
    lhsi = np.zeros(rhsi.shape).astype(int)
    for ind in range(len(rhsi)):
        #greatest value in lhsi_not_sorted thats less than rhsi[ind]
        
        _lind = np.argwhere(lhsi_not_sorted < rhsi[ind]).flatten()[-1]
        lhsi[ind] = lhsi_not_sorted[_lind]
        lhsi_not_sorted[_lind] = 15
        
    return c, k, lhsi, rhsi
c, k, lhsi, rhsi = get_constants(data)
w = np.zeros(14) + 9
print(c, k, '\n', lhsi, rhsi)

[ 12  11  13  11  14 -10  11  -9  -3  13  -5 -10  -4  -5] [ 4 11  5 11 14  7 11  4  6  5  9 12 14 14] 
 [4 6 3 9 2 1 0] [ 5  7  8 10 11 12 13]


In [5]:
print("i:", end='')
[print(f"{x: 4d}", end='') for x in range(0, 14)]
print("\nc:", end='')
[print(f"{x: 4d}", end='') for x in c]
print("\nk:", end='')
[print(f"{x: 4d}", end='') for x in k]
":)"

i:   0   1   2   3   4   5   6   7   8   9  10  11  12  13
c:  12  11  13  11  14 -10  11  -9  -3  13  -5 -10  -4  -5
k:   4  11   5  11  14   7  11   4   6   5   9  12  14  14

':)'

In [6]:
b = k[lhsi] + c[rhsi] # w(rhs) - w(lhs)
wlhs = np.ones(7) * 9 # to find the highest number, set the most significant side to 9
wrhs = b + wlhs
wlhs[wrhs > 9] -= wrhs[wrhs>9]-9 # make sure neither number is >9
wrhs = b + wlhs
w = np.ones(14, dtype=int) * 9
w[lhsi] = wlhs
w[rhsi] = wrhs
print("part 1: " + "".join(w.astype(str)))
print("tested:", test_model("".join(w.astype(str))))

part 1: 92915979999498
tested: (0, 0, 0, 0)


In [7]:
wlhs = np.ones(7) #same as above, but not minimizing while keeping numbers from being too small
wrhs = b + wlhs
wlhs[wrhs < 1] += 1-wrhs[wrhs<1]
wrhs = b + wlhs
w = np.ones(14, dtype=int) * 9
w[lhsi] = wlhs
w[rhsi] = wrhs
print("part 1: " + "".join(w.astype(str)))
print("tested:", test_model("".join(w.astype(str))))

part 1: 21611513911181
tested: (0, 0, 0, 0)
